# Semester 3 Coding Portfolio Topic 5 Formative Part 1/2:
# Social network analysis 1: Create a network from text data and analyze it

This notebook covers the following topics:
 - Extracting interaction data
 - Creating networkx graphs
 - Exporting for visual network analysis

This notebook is expected to take around 5 hours to complete:
 - 2 hours for the formative part
 - 3 hours of self-study on the topics covered by this notebook

Like all topics in this portfolio, this topic is split into two sections:
 - Formative 
 - Summative

<b>Formative section</b><br>
Simply complete the given functions such that they pass the automated tests. This part is graded Pass/Fail; you must get 100% correct!
You can submit your notebook through Canvas as often as you like. Make sure to start doing so early to insure that your code passes all tests!
You may ask for help from fellow students and TAs on this section, and solutions might be provided later on.

In this workshop, we will use networkx to create a network from text data, open it in Gephi, and then use networkx to analyze its structure.


### Part 1. Extract interactions from textual data

In this exercise, we will go from data in a dataframe to creating a network that we can analyze, and that we can open with Gephi.

In your own project, you are unlikely to come across ready-to-process data in a format that you can open with networkx or Gephi. Instead, you need to create network from your own data.

In this case, we will look at a classic dataset: the Enron emails. The Enron email dataset is a large collection of email data from mostly senior management of the Enron Corporation. It was made public by the Federal Energy Regulatory Commission during its investigation after Enron's collapse in 2001. The dataset contains roughly half a million emails organized into folders. It's one of the largest datasets of real-world emails available for public study and has been widely used for research in various fields like social network analysis, natural language processing, and machine learning.

The Enron dataset is particularly notable for its insights into corporate communications and behaviors, and it has played a significant role in advancing the study of information processing and management. Due to its real-world nature, including the variety of topics and the informal style of communication, the dataset presents unique challenges and opportunities for data analysis and has become a benchmark in the study of electronic communications.

We will here use the dataset to create a social network of emails between individuals, and we will analyze the structure of this network.

The Enron case led to the conviction of two managers: Kenneth Lay (kenneth.lay@enron.com) and Jeffrey Skilling (jeffreyskilling@yahoo.com). We will examine the centrality of these nodes in the network, and identify the community to which they belong. 

To make the analysis more feasible, we will focus on emails in December 2001 - the month when the company went under.

In [1]:
# Imports
# Install all necessary packages first to avoid kernel restart
# This ensures all dependencies are available before running the notebook
!pip3 install networkx python-louvain pandas matplotlib -q

import re
import json
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import community as community_louvain

In [2]:
df = pd.read_csv('sem3_topic5_sna_formative1_data.csv.gz')
df.head()

,message
0,Message-ID: <18782981.1075855378110.JavaMail.e...
1,Message-ID: <15464986.1075855378456.JavaMail.e...
2,Message-ID: <7391389.1075855378477.JavaMail.ev...
3,Message-ID: <8572706.1075855378498.JavaMail.ev...
4,Message-ID: <32300323.1075855378519.JavaMail.e...


As you can see, the email is just a long text message. We need to parse out the actual data and email addresses from the text using regexps.

In [3]:
print(df.sample(1).message.values[0])

Message-ID: <24743962.1075858729372.JavaMail.evans@thyme>
Date: Tue, 23 Oct 2001 19:27:42 -0700 (PDT)
From: expediatraveldeals_029715@expedia.customer-email.com
To: rshapiro@enron.com
Subject: Get away -- and save up to 30% on your vacation!
Mime-Version: 1.0
Content-Type: text/plain; charset=ANSI_X3.4-1968
Content-Transfer-Encoding: 7bit
X-From: Expedia Travel Deals <ExpediaTravelDeals_029715@expedia.customer-email.com>
X-To: rshapiro@enron.com
X-cc: 
X-bcc: 
X-Folder: \RSHAPIRO (Non-Privileged)\Shapiro, Richard\Deleted Items
X-Origin: Shapiro-R
X-FileName: RSHAPIRO (Non-Privileged).pst

 <http://www.expedia.com/default.asp?rfrr=-1412>	 
	home <http://www.expedia.com/pubspec/scripts/eap.asp?GOTO=HOME&rfrr=-1412>flights <http://www.expedia.com/pubspec/scripts/eap.asp?GOTO=FLIGHTLAUNCH&rfrr=-1412>hotels <http://www.expedia.com/pubspec/scripts/eap.asp?GOTO=HOTLAUNCH&rfrr=-1412>cars <http://www.expedia.com/pub/agent.dll?qscr=carw&rfrr=-1412>vacations <http://www.expedia.com/pubspec/script

In [4]:
def extract_subject(message):
    # Regular expression pattern to match the subject line
    pattern = r'^Subject: (.+)$'

    # Search for the pattern in the message, using multiline mode
    match = re.search(pattern, message, re.MULTILINE)

    # Return the subject if a match is found, else return None
    return match.group(1) if match else None

# Example: this function extracts a date from the email
def extract_date(message):
    match = re.search(r"Date: .*?, (\d+ \w+ \d{4})", message)
    return match.group(1) if match else None

# Extract and convert date
df['date'] = df['message'].apply(extract_date)
df['date'] = pd.to_datetime(df['date'], errors='coerce', format='%d %b %Y')

df['subject'] = df['message'].apply(extract_subject)

#Your task is to extract from and to email, and to put these as new columns "from_email" and "to_email"
# Regular expression patterns for "From", "To" email addresses. We ignore CC here for simplicity
from_pattern = r"From: (\S+@\S+)"
to_pattern = r"To: ([\w\.,@]+)"


### Exercise 1A

In [5]:
def extract_email_addresses(dataframe):
    """Extract from and to email addresses from message column and add them as new columns.
    
    Args:
        dataframe: The dataframe containing the message column with email data.
    
    Returns:
        Filtered dataframe with from_email and to_email columns containing only valid emails.
    """
    
    # TODO 1: Write functions to extract from_email and to_email from the message text using regex patterns
    
    # Define a regex pattern to validate email addresses
    # This pattern ensures emails have the format: username@domain.extension
    # It checks for alphanumeric characters, dots, underscores, plus signs, and hyphens
    email_validation_pattern = re.compile(r"^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$")
    
    # Helper function to extract "From" email address from a message
    # Uses the from_pattern regex to find the email after "From: "
    def extract_from_email(message):
        if pd.isna(message):  # Check if message is NaN
            return None
        match = re.search(from_pattern, message)  # Search for the pattern in the message
        if match:
            email = match.group(1)  # Extract the first captured group (the email)
            # Validate the email format - only return if it matches valid email pattern
            if email_validation_pattern.match(email):
                return email
        return None
    
    # Helper function to extract "To" email address from a message
    # The "To" field might contain multiple emails or be formatted differently
    def extract_to_email(message):
        if pd.isna(message):  # Check if message is NaN
            return None
        match = re.search(to_pattern, message)  # Search for the pattern in the message
        if match:
            email_str = match.group(1)  # Extract the first captured group
            
            # The "To" field might contain multiple emails separated by commas
            # Split by comma and take the first email address
            emails = [e.strip() for e in email_str.split(',')]
            
            # Try to find a valid email in the list
            for email in emails:
                # Clean up the email (remove any extra whitespace)
                email = email.strip()
                # Validate the email format
                if email_validation_pattern.match(email):
                    return email
        return None
    
    # Apply the extraction functions to each message in the dataframe
    # This creates new columns 'from_email' and 'to_email' with the extracted addresses
    dataframe['from_email'] = dataframe['message'].apply(extract_from_email)
    dataframe['to_email'] = dataframe['message'].apply(extract_to_email)
    
    # Filter the dataframe to only keep rows where both from_email and to_email are valid (not None)
    # This ensures we only work with complete email pairs
    dataframe = dataframe.dropna(subset=['from_email', 'to_email'])
    
    return dataframe

df = extract_email_addresses(df)

# BEGIN TESTS
# @name("email_columns_created")
# @description("Check that email extraction created valid email columns")
def test_email_columns_created():
    assert 'from_email' in df.columns, "Missing column from_email"
    assert 'to_email' in df.columns, "Missing column to_email"
    assert df['from_email'].notna().any(), "No from_email values found"
    assert df['to_email'].notna().any(), "No to_email values found"
test_email_columns_created()

# @name("email_format_valid")
# @description("Check that all emails match valid email pattern after filtering")
def test_email_format_valid():
    email_re = re.compile(r"^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$")
    # All emails should match the valid email pattern (filtering was applied)
    assert df['from_email'].dropna().apply(lambda x: bool(email_re.match(x))).all(), "Invalid from_email found - filtering failed"
    assert df['to_email'].dropna().apply(lambda x: bool(email_re.match(x))).all(), "Invalid to_email found - filtering failed"
test_email_format_valid()

# @name("dataframe_filtered")
# @description("Check that DataFrame has valid data after filtering")
def test_dataframe_filtered():
    assert 245000 <=len(df) <= 246000, "DataFrame is not filtered correctly"
test_dataframe_filtered()
# END TESTS

### Exercise 1B

Next step: Now that we have the date and the from and to emails, we want to aggregate on the from and to, so that we get the number of emails sent between the two emails.

The result should be a dataframe with columns:

from_email | to_email  | count

In [6]:
def create_email_edges(dataframe):
    """Aggregate emails by from_email and to_email to count the number of emails between each pair.
    
    Args:
        dataframe: The dataframe containing from_email and to_email columns.
    
    Returns:
        A dataframe with columns: from_email, to_email, count
    """
    
    # TODO 2: Group by from_email and to_email, then count occurrences
    
    # Group the dataframe by 'from_email' and 'to_email' columns
    # This groups all rows that have the same sender-receiver pair together
    grouped = dataframe.groupby(['from_email', 'to_email'])
    
    # Count the number of emails in each group using the size() method
    # This gives us the number of emails sent from each 'from_email' to each 'to_email'
    # The result is a Series with a MultiIndex (from_email, to_email) and values as counts
    email_counts = grouped.size()
    
    # Reset the index to convert the MultiIndex back into regular columns
    # This creates a DataFrame with columns: from_email, to_email, and the count (default name is 0)
    edges = email_counts.reset_index()
    
    # Rename the count column to 'count' for clarity
    # The default name after reset_index() is 0, so we rename it
    edges = edges.rename(columns={0: 'count'})
    
    return edges

email_edges = create_email_edges(df)
print(email_edges)

# BEGIN TESTS
# @name("email_edges_structure")
# @description("Check that email_edges DataFrame has correct structure")
def test_email_edges_structure():
    assert isinstance(email_edges, pd.DataFrame), "email_edges should be a DataFrame"
    expected_cols = {'from_email', 'to_email', 'count'}
    assert expected_cols.issubset(email_edges.columns), f"Missing columns. Expected {expected_cols}, got {set(email_edges.columns)}"
test_email_edges_structure()

# @name("email_edges_aggregation_correctness")
# @description("Check that aggregation produced valid counts without duplicates")
def test_email_edges_aggregation_correctness():
    # Counts should be positive integers
    assert pd.api.types.is_numeric_dtype(email_edges['count']), "Column 'count' should be numeric"
    assert (email_edges['count'] > 0).all(), "All counts should be positive"
    # No duplicate from-to pairs (proper grouping)
    assert not email_edges[['from_email', 'to_email']].duplicated().any(), "Should not have duplicate from_email-to_email pairs (groupby failed)"
test_email_edges_aggregation_correctness()

# @name("email_edges_count_values")
# @description("Check that count values are correct")
def test_email_edges_count_values():
    # Total count in email_edges should be less than or equal to original df length
    assert 46900 <= len(email_edges) <= 47000, "email_edges DataFrame does not have enough rows"
    assert email_edges['count'].sum() <= len(df), "Sum of counts should not exceed original dataframe length"
test_email_edges_count_values()
# END TESTS

                                  from_email                  to_email  count
0             --migrated--bmishkin@ercot.com      mockmarket@ercot.com      1
1                --migrated--dodle@ercot.com             set@ercot.com      1
2                         -nikole@excite.com   bill.williams@enron.com      8
3        -persson@ricemail.ricefinancial.com  barry.tycholiz@enron.com      5
4      01019@salespoint.dealerconnection.com     jason.wolfe@enron.com      4
...                                      ...                       ...    ...
46933                     zufferli@enron.com   john.lavorato@enron.com      2
46934                 zulie.flores@enron.com   marla.barnard@enron.com      3
46935                    zwharton@dawray.com   martin.cuilla@enron.com      6
46936                    zwharton@dawray.com         mcuilla@enron.com      3
46937                       zzmacmac@aol.com      j.kaminski@enron.com      2

[46938 rows x 3 columns]


## Part 2: Create networkx graph from data 
In the next step, we're going to create a networkx graph from this network, and save to have a look at the graph in Gephi.

In [7]:
#Now it's time to create the network! 
G = nx.Graph()

# Two people were convicted in the Enron scandal: Kenneth Lay, and Jeffrey Skilling. 
# The nodes should have a property 'convicted'. If the email address is either 'kenneth.lay@enron.com' or 'jeffreyskilling@yahoo.com', convicted == True, otherwise False.

# Use G.add_node(id, additional_data="data") to add nodes
# Use G.add_edge(from_id,to_id, weight=5) to add edges



### Exercise 2A

In [8]:
def build_network(graph, edges_df):
    """Build a network graph from email edges with convicted attribute for specific nodes.
    
    Args:
        graph: Empty NetworkX graph object.
        edges_df: DataFrame with columns from_email, to_email, count.
    
    Returns:
        Graph with nodes (having convicted attribute) and weighted edges.
    """
    
    # TODO 3: Create nodes with 'convicted' attribute and add edges with weights
    
    # Define the email addresses of the two convicted executives from the Enron scandal
    # These are the individuals we want to mark with the 'convicted' attribute
    convicted_emails = {'kenneth.lay@enron.com', 'jeffreyskilling@yahoo.com'}
    
    # First, collect all unique email addresses that appear in either from_email or to_email columns
    # This gives us all the nodes (people) in our network
    all_emails = set(edges_df['from_email'].unique()) | set(edges_df['to_email'].unique())
    
    # Add each email address as a node in the graph
    # For each node, we set the 'convicted' attribute:
    # - True if the email is one of the convicted executives
    # - False otherwise
    for email in all_emails:
        is_convicted = email in convicted_emails  # Check if this email belongs to a convicted executive
        graph.add_node(email, convicted=is_convicted)  # Add node with the convicted attribute
    
    # Now add edges between nodes based on the email interactions
    # Iterate through each row in the edges_df DataFrame
    for _, row in edges_df.iterrows():
        from_email = row['from_email']  # Get the sender's email
        to_email = row['to_email']      # Get the receiver's email
        weight = row['count']           # Get the number of emails (this becomes the edge weight)
        
        # Add an edge between from_email and to_email with the count as the weight
        # The weight represents how many emails were exchanged between these two people
        # NetworkX will automatically create the edge if it doesn't exist, or update it if it does
        graph.add_edge(from_email, to_email, weight=weight)
    
    return graph

G = build_network(G, email_edges)

# BEGIN TESTS
# @name("graph_has_nodes")
# @description("Check that graph has correct number of nodes")
def test_graph_has_nodes():
    assert G.number_of_nodes() >= 20000, "Graph does not have correct number of nodes"
test_graph_has_nodes()

# @name("graph_has_edges")
# @description("Check that graph has correct number of edges")
def test_graph_has_edges():
    assert G.number_of_edges() > 30000, "Graph does not have enough edges"
test_graph_has_edges()

# @name("convicted_attribute_exists")
# @description("Check that nodes have convicted attribute")
def test_convicted_attribute_exists():
    sample_node = list(G.nodes())[0]
    assert 'convicted' in G.nodes[sample_node], "Nodes missing 'convicted' attribute"
test_convicted_attribute_exists()

# @name("convicted_values")
# @description("Check that nodes have correct convicted attribute values")
def test_convicted_values():
    kenneth_exists = 'kenneth.lay@enron.com' in G.nodes()
    jeffrey_exists = 'jeffreyskilling@yahoo.com' in G.nodes()
    if kenneth_exists:
        assert G.nodes['kenneth.lay@enron.com']['convicted'] == True, "kenneth.lay@enron.com should have convicted=True"
    if jeffrey_exists:
        assert G.nodes['jeffreyskilling@yahoo.com']['convicted'] == True, "jeffreyskilling@yahoo.com should have convicted=True"
    
    # Check that only 2 executives are marked as convicted
    convicted_count = sum(1 for node in G.nodes() if G.nodes[node].get('convicted', False))
    assert convicted_count == 2, "There should be exactly 2 convicted nodes"
test_convicted_values()

# @name("edge_weights")
# @description("Check that edges have weight attribute")
def test_edge_weights():
    sample_edge = list(G.edges(data=True))[0]
    assert 'weight' in sample_edge[2], "Edges missing 'weight' attribute"
test_edge_weights()
# END TESTS

In [9]:
len(G.nodes())

20157

In [10]:
# You can try to plot it - but you might find that the network is too big and that your python loads forever!

# # When we have our network, we can plot it
# # nx.draw(G) plots - but not very pretty
# # The following code is more customizable 

# def plot_network(G):
#     # Use a layout that spreads out the nodes, like spring_layout
#     pos = nx.spring_layout(G, scale=2)
    
#     # Draw the network
#     plt.figure(figsize=(8, 8))  # Large figure size
    
#     #We plot the convicted individuals with red.
#     node_color = ['red' if G.nodes[node].get('convicted', False) else 'blue' for node in G]
#     nx.draw_networkx_nodes(G, pos, node_size=5, node_color=node_color)
#     nx.draw_networkx_edges(G, pos, width=0.1, alpha=0.5)
    
#     # Set plot limits to avoid cutting off nodes
#     plt.xlim([-1, 1])
#     plt.ylim([-1, 1])
    
#     # Turn off axis
#     plt.axis('off')
    
#     # Show plot
#     plt.show()
# plot_network(G)

### Exercise 2B: Getting the giant component 
As you can see, the resulting network has a lot of small components that are not connected to the larger network. Often it makes sense to focus only on the giant component: that is, the biggest connected component in the network.

To get the components of the graph, use nx.connected_components(G)

Remember that the giant component is the largest component.

Once you've identified it, you can use G.subgraph(component) to turn the nodes into a graph

Put the result back in the variable G.

In [11]:
def extract_giant_component(graph):
    """Extract the giant component (largest connected component) from the graph.
    
    Args:
        graph: NetworkX graph object.
    
    Returns:
        Subgraph containing only the giant component.
    """
    
    # TODO 4: Find the largest connected component and return it as a subgraph
    
    # Get all connected components in the graph
    # A connected component is a set of nodes where you can reach any node from any other node
    # In a network, there might be multiple disconnected groups of nodes
    # nx.connected_components() returns a generator of sets, where each set contains nodes in one component
    components = nx.connected_components(graph)
    
    # Find the largest component (the "giant component")
    # This is typically the main network we want to analyze, as it contains most of the nodes
    # We use max() with key=len to find the component with the most nodes
    giant_component_nodes = max(components, key=len)
    
    # Create a subgraph containing only the nodes in the giant component
    # G.subgraph() creates a new graph that includes only the specified nodes and their edges
    # This effectively filters out all the smaller, disconnected components
    graph = graph.subgraph(giant_component_nodes)
    
    return graph

G = extract_giant_component(G)

# BEGIN TESTS
# @name("graph_is_connected")
# @description("Check that G is connected (giant component)")
def test_graph_is_connected():
    assert nx.is_connected(G), "Graph G should be connected (giant component)"
test_graph_is_connected()

# @name("giant_component_size")
# @description("Check that giant component has correct size")
def test_giant_component_size():
    assert G.number_of_nodes() > 17000, "Giant component does not have correct number of nodes"
test_giant_component_size()

# @name("giant_component_edges")
# @description("Check that giant component has correct number of edges")
def test_giant_component_edges():
    assert G.number_of_edges() >  30000, "Giant component does not have correct number of edges"
test_giant_component_edges()
# END TESTS

In [12]:
#The number of nodes in our new graph
len(G.edges())

38262

In [13]:
# Might crash your python!
# plot_network(G)

## Part 3: Save the network, open with Gephi, and carry out Visual Network Analysis
We can now save the network as a file, so that we can open it with Gephi and analyze it.

There are several fileformats that we can save in, that are compatible with Gephi. GEXF and GraphML are two common formats. You can read more about the formats and their pros/cons on this website: https://networkx.org/documentation/stable/reference/readwrite/index.html 



In [14]:
# To visualize, networkx works poorly. 
# Let's look in gephi! 
nx.write_gexf(G,'enron_network.gexf')

In [15]:
# If the network is too large to deal with on your laptop, you might want to filter out small nodes.

# Filter nodes with degree less than N
N = 3
filtered_nodes = [node for node, degree in G.degree() if degree >= N]

# Create a new graph with the filtered nodes
filtered_graph = G.subgraph(filtered_nodes).copy()

Now open this file with Gephi and use it to create a better plot of the network!

See if you can figure out the different communities, and what charaterizes them. Feel free to use google to search the emails and names!

## Part 4: Use networkx to analyze the convicted bankers
We're now going to use networkx to get some exact measures on the network, and to measure the centrality and influence of the two convicted bankers.

In [16]:
# Normally, you would calculate these values using the following functions. You can uncomment them to try it out yourself.
# pagerank = nx.pagerank(G)
# betweenness = nx.betweenness_centrality(G)
# degree_centrality = nx.degree_centrality(G)
# eigenvector = nx.eigenvector_centrality(G)

# However, this takes a long time, so we've also prepared these values for you here:
with open('sem3_topic5_sna_formative1_data2.json', 'r') as f:
    data = json.load(f)
pagerank = data['pagerank']
betweenness = data['betweenness']
degree_centrality = data['degree_centrality']
eigenvector = data['eigenvector']

In [17]:
# Your output should look as follows:

# PageRank
# Kenneth lay is the number 2 most central.
# Jeffrey Skilling is the number 2478 most central

# Betweenness
# Kenneth lay is the number 6 most central.
# Jeffrey Skilling is the number 2158 most central

# Degree
# Kenneth lay is the number 2 most central.
# Jeffrey Skilling is the number 2221 most central

# Eigenvector
# Kenneth lay is the number 21 most central.
# Jeffrey Skilling is the number 1555 most central

### Exercise 3A

In [18]:
def get_rank(centrality_dict):
    """Print the rank of Kenneth Lay and Jeffrey Skilling in a given centrality dictionary.
    
    Args:
        centrality_dict: Dictionary mapping node names to centrality scores.
    
    Returns:
        Tuple of (kenneth_rank, jeffrey_rank) where each is an integer rank or None if not found.
    """
    
    # TODO 5: Iterate through sorted centrality scores and find the rank of the two convicted individuals
    
    # Define the email addresses of the two convicted executives we want to rank
    kenneth_email = 'kenneth.lay@enron.com'
    jeffrey_email = 'jeffreyskilling@yahoo.com'
    
    # Sort the centrality dictionary by values in descending order
    # Centrality measures typically have higher values for more central/important nodes
    # sorted() with reverse=True gives us nodes from most central to least central
    # The result is a list of tuples: (node_name, centrality_score)
    sorted_nodes = sorted(centrality_dict.items(), key=lambda x: x[1], reverse=True)
    
    # Initialize variables to store the ranks (1-indexed, where 1 is most central)
    kenneth_rank = None
    jeffrey_rank = None
    
    # Iterate through the sorted list with enumerate to get both the index (rank) and the node data
    # We use enumerate starting at 1 because ranks are typically 1-indexed (1st, 2nd, 3rd, etc.)
    for rank, (node, score) in enumerate(sorted_nodes, start=1):
        # Check if this node is Kenneth Lay
        if node == kenneth_email:
            kenneth_rank = rank  # Store his rank
        # Check if this node is Jeffrey Skilling
        if node == jeffrey_email:
            jeffrey_rank = rank  # Store his rank
    
    # Print the results in a readable format
    # If a person is found, print their rank; otherwise indicate they weren't found
    if kenneth_rank is not None:
        print(f"Kenneth lay is the number {kenneth_rank} most central.")
    else:
        print("Kenneth lay was not found in the centrality dictionary.")
    
    if jeffrey_rank is not None:
        print(f"Jeffrey Skilling is the number {jeffrey_rank} most central.")
    else:
        print("Jeffrey Skilling was not found in the centrality dictionary.")
    
    # Return the ranks as a tuple for potential use in tests or further analysis
    return (kenneth_rank, jeffrey_rank)

print("PageRank")
get_rank(pagerank)
print("\nBetweenness")
get_rank(betweenness)
print("\nDegree")
get_rank(degree_centrality)
print("\nEigenvector")
get_rank(eigenvector)

# BEGIN TESTS
# @name("get_rank_sorting")
# @description("Check that get_rank function correctly sorts and ranks nodes")
def test_get_rank_sorting():
    test_dict = {'a': 0.5, 'kenneth.lay@enron.com': 0.9, 'b': 0.3, 'jeffreyskilling@yahoo.com': 0.1}
    kenneth_rank, jeffrey_rank = get_rank(test_dict)
    assert kenneth_rank == 1, "Kenneth Lay ranking is incorrect"
    assert jeffrey_rank == 4, "Jeffrey Skilling ranking is incorrect"
test_get_rank_sorting()

# @name("get_rank_pagerank")
# @description("Check that Kenneth Lay and Jeffrey Skilling have correct PageRank ranks")
def test_get_rank_pagerank():
    if 'kenneth.lay@enron.com' in pagerank:
        kenneth_rank, jeffrey_rank = get_rank(pagerank)
        assert kenneth_rank is not None and kenneth_rank < 30, "Kenneth Lay PageRank ranking is incorrect"
        if jeffrey_rank is not None:
            assert jeffrey_rank > 5000, "Jeffrey Skilling PageRank ranking is incorrect"
test_get_rank_pagerank()

# @name("get_rank_betweenness")
# @description("Check that Kenneth Lay and Jeffrey Skilling have correct Betweenness ranks")
def test_get_rank_betweenness():
    if 'kenneth.lay@enron.com' in betweenness:
        kenneth_rank, jeffrey_rank = get_rank(betweenness)
        assert kenneth_rank is not None and kenneth_rank < 20, "Kenneth Lay Betweenness ranking is incorrect"
        if jeffrey_rank is not None:
            assert jeffrey_rank > 5000, "Jeffrey Skilling Betweenness ranking is incorrect"
test_get_rank_betweenness()

# @name("get_rank_degree")
# @description("Check that Kenneth Lay and Jeffrey Skilling have correct Degree Centrality ranks")
def test_get_rank_degree():
    if 'kenneth.lay@enron.com' in degree_centrality:
        kenneth_rank, jeffrey_rank = get_rank(degree_centrality)
        assert kenneth_rank is not None and kenneth_rank < 20, "Kenneth Lay Degree Centrality ranking is incorrect"
        if jeffrey_rank is not None:
            assert jeffrey_rank > 5000, "Jeffrey Skilling Degree Centrality ranking is incorrect"
test_get_rank_degree()

# @name("get_rank_eigenvector")
# @description("Check that Kenneth Lay and Jeffrey Skilling have correct Eigenvector Centrality ranks")
def test_get_rank_eigenvector():
    if 'kenneth.lay@enron.com' in eigenvector:
        kenneth_rank, jeffrey_rank = get_rank(eigenvector)
        assert kenneth_rank is not None and kenneth_rank < 30, "Kenneth Lay Eigenvector Centrality ranking is incorrect"
        if jeffrey_rank is not None:
            assert jeffrey_rank > 5000, "Jeffrey Skilling Eigenvector Centrality ranking is incorrect"
test_get_rank_eigenvector()
# END TESTS

PageRank
Kenneth lay is the number 11 most central.
Jeffrey Skilling is the number 10213 most central.

Betweenness
Kenneth lay is the number 3 most central.
Jeffrey Skilling is the number 8780 most central.

Degree
Kenneth lay is the number 3 most central.
Jeffrey Skilling is the number 9473 most central.

Eigenvector
Kenneth lay is the number 12 most central.
Jeffrey Skilling is the number 6554 most central.
Kenneth lay is the number 1 most central.
Jeffrey Skilling is the number 4 most central.
Kenneth lay is the number 11 most central.
Jeffrey Skilling is the number 10213 most central.
Kenneth lay is the number 3 most central.
Jeffrey Skilling is the number 8780 most central.
Kenneth lay is the number 3 most central.
Jeffrey Skilling is the number 9473 most central.
Kenneth lay is the number 12 most central.
Jeffrey Skilling is the number 6554 most central.


### 2. Find their communities
The second part is to use the Louvain community detection to identify which community the two belong to. 

We will then put the identified communities as a column in the original dataframe, allowing us to potentially carry out text analysis to identify what characterizes the communities.

In [19]:
# Detect communities using the Louvain method
partition = community_louvain.best_partition(G)

# Calculate the modularity
modularity = community_louvain.modularity(partition, G)

print("Modularity:", modularity)


Modularity: 0.7657287229010155


### Exercise 3B
Which communities are our two convicts in?

Use the partition to create a new column in the original df called "partition" that contains the partition of the individual associated to the email!

In [20]:
def add_partition_columns(dataframe, partition_dict):
    """Add partition and partition_size columns to the dataframe based on Louvain communities.
    
    Args:
        dataframe: The original dataframe with from_email column.
        partition_dict: Dictionary mapping email addresses to partition/community IDs.
    
    Returns:
        Dataframe with added partition and partition_size columns.
    """
    
    # TODO 6: Create partition column mapping from_email to community ID, and partition_size with community size
    
    # First, we need to calculate the size of each community/partition
    # We'll count how many nodes (email addresses) belong to each partition ID
    # This will help us understand how large each community is
    
    # Import Counter to count occurrences efficiently
    from collections import Counter
    
    # Count how many nodes belong to each partition ID
    # Counter creates a dictionary where keys are partition IDs and values are counts
    partition_sizes = Counter(partition_dict.values())
    
    # Create the 'partition' column by mapping each from_email to its community ID
    # We use the map() function to look up each email address in the partition_dict
    # If an email is not in the partition_dict (e.g., it wasn't in the graph), it will be NaN
    dataframe['partition'] = dataframe['from_email'].map(partition_dict)
    
    # Create the 'partition_size' column by mapping each partition ID to its size
    # We use the partition column we just created and map each partition ID to its size
    # This tells us how many people are in the same community as each email sender
    dataframe['partition_size'] = dataframe['partition'].map(partition_sizes)
    
    return dataframe

df = add_partition_columns(df, partition)

# BEGIN TESTS
# @name("partition_column_exists")
# @description("Check that partition column was created")
def test_partition_column_exists():
    assert 'partition' in df.columns, "Missing column 'partition'"
test_partition_column_exists()

# @name("partition_size_column_exists")
# @description("Check that partition_size column was created")
def test_partition_size_column_exists():
    assert 'partition_size' in df.columns, "Missing column 'partition_size'"
test_partition_size_column_exists()

# @name("partition_values")
# @description("Check that partition column has valid values")
def test_partition_values():
    assert df['partition'].notna().any(), "partition column has no non-null values"
test_partition_values()

# @name("partition_size_numeric")
# @description("Check that partition_size values are numeric")
def test_partition_size_numeric():
    assert pd.api.types.is_numeric_dtype(df['partition_size']), "partition_size should be numeric"
test_partition_size_numeric()

# @name("partition_values_match_dict")
# @description("Check that partition column values correctly match partition_dict")
def test_partition_values_match_dict():
    from collections import Counter
    # Check that partition values match the partition_dict for emails that exist in both
    for idx, row in df.iterrows():
        email = row['from_email']
        if email in partition:
            expected_partition = partition[email]
            actual_partition = row['partition']
            assert actual_partition == expected_partition, f"Partition value mismatch for {email}: expected {expected_partition}, got {actual_partition}"
test_partition_values_match_dict()

# @name("partition_size_correctness")
# @description("Check that partition_size values correctly reflect community sizes")
def test_partition_size_correctness():
    from collections import Counter
    # Calculate expected partition sizes from partition_dict
    partition_counts = Counter(partition.values())
    # Check that partition_size matches the count of nodes in each partition
    for idx, row in df.iterrows():
        if pd.notna(row['partition']):
            expected_size = partition_counts[row['partition']]
            actual_size = row['partition_size']
            assert actual_size == expected_size, f"Partition size mismatch for partition {row['partition']}: expected {expected_size}, got {actual_size}"
test_partition_size_correctness()
# END TESTS

In [21]:
# We can then look at the common subjects for the emails, maybe applying TF-IDF
df.loc[~df['subject'].isna()].groupby(['partition'])['subject'].agg(lambda x: " ".join(x))

partition
0.0     Mid-Year 2001 Performance Feedback Another ENA...
1.0     NEWS Deadline We need news! Expense Market Ins...
2.0     Your Approval is Overdue: Access Request for m...
3.0     Trading in ERCOT Fwd: FW: sooo true Position F...
4.0     Re: Denver trading Win A 2002 Porsche Boxster!...
5.0     Yahoo! Newsletter, May 2001 Freidman, Billings...
6.0     New Active X for EOL Website Follow up:   New ...
7.0     {Resend} E234 Options/Derivatives notes Ventur...
8.0     Credit Watch List--Week of 10/22/01 Credit Wat...
9.0     Undeliverable: Delivery Status Notification (F...
10.0    test mail ClickAtHome Order Verification Fw: A...
11.0    Welcome to CustomClips, a Service of Dow Jones...
12.0    Conference Call Today with FERC Staff Comments...
13.0    RSVP REQUESTED - Emissions Strategy Meeting......
14.0     "Save the Date" - Associate / Analyst Program...
15.0    Quarterly Managing Director Meeting - Monday, ...
16.0    BCP Seat Assignments WPS Yarmouth/Wyman UE off...
17.0